In [1]:
import pandas as pd
import numpy as np

sales_raw = pd.read_csv("./sales_raw.csv")
discounts_raw = pd.read_csv("./discounts_raw.csv")

sales_raw['date'] = pd.to_datetime(sales_raw['date'])
discounts_raw['date_start'] = pd.to_datetime(discounts_raw['date_start'])
discounts_raw['date_end'] = pd.to_datetime(discounts_raw['date_end'])

# Сортировка данных по дате для использования merge_asof
sales_raw = sales_raw.sort_values('date')
discounts_raw = discounts_raw.sort_values('date_start')

# Присоединение скидок к продажам
merged_data = pd.merge_asof(sales_raw, discounts_raw, left_on='date', right_on='date_start', by='item_id', direction='backward')

# Удаление скидок, которые уже закончились
merged_data = merged_data[merged_data['date'] <= merged_data['date_end']]


In [2]:
merged_data.head()

,date,item_id,qnty,promo_typeCode,sale_price_before_promo,sale_price_time_promo,date_start,date_end
64102,2020-01-02,100551,7.0,7.0,139.9,85.0,2020-01-02,2020-01-08
64103,2020-01-02,100553,1.0,7.0,139.9,85.0,2020-01-02,2020-01-08
64110,2020-01-02,100454,2.0,5.0,59.9,39.9,2020-01-02,2020-01-08
64113,2020-01-02,100370,23.0,5.0,339.9,169.9,2020-01-02,2020-01-08
64116,2020-01-02,100545,43.0,5.0,159.9,99.9,2020-01-02,2020-01-08


In [3]:
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error

# Создание признаков
merged_data['day_of_week'] = merged_data['date'].dt.dayofweek
merged_data['is_discount'] = ~merged_data['promo_typeCode'].isna()

# Заполнение пропусков
merged_data['sale_price_before_promo'].fillna(merged_data['sale_price_before_promo'].mean(), inplace=True)
merged_data['sale_price_time_promo'].fillna(merged_data['sale_price_time_promo'].mean(), inplace=True)
merged_data['promo_typeCode'].fillna(0, inplace=True)
merged_data['qnty'].fillna(0, inplace=True)

# Использование данных до декабря 2023 года для обучения модели
train_data = merged_data[merged_data['date'] < '2023-12-01']
test_data = merged_data[(merged_data['date'] >= '2023-12-01') & (merged_data['date'] <= '2023-12-31')]

X_train = train_data[['item_id', 'day_of_week', 'is_discount', 'sale_price_before_promo', 'sale_price_time_promo']]
y_train = train_data['qnty']
X_test = test_data[['item_id', 'day_of_week', 'is_discount', 'sale_price_before_promo', 'sale_price_time_promo']]
y_test = test_data['qnty']

# Модель бустинга
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Прогнозирование на данных за декабрь 2023 года
y_pred = model.predict(X_test)

# Оценка качества прогноза
rmse = root_mean_squared_error(y_test, y_pred)
print(f'RMSE на данных за декабрь 2023 года: {rmse}')

# Подготовка данных для прогноза на январь 2024 года
january_dates = pd.date_range(start='2024-01-01', end='2024-01-31')
items_in_december = test_data['item_id'].unique()

# Создание датафрейма для прогноза
prediction_data = pd.DataFrame([(date, item) for date in january_dates for item in items_in_december], columns=['date', 'item_id'])
prediction_data['day_of_week'] = prediction_data['date'].dt.dayofweek

# Присоединение информации о скидках
prediction_data = pd.merge_asof(prediction_data.sort_values('date'), discounts_raw, left_on='date', right_on='date_start', by='item_id', direction='backward')
prediction_data = prediction_data[prediction_data['date'] <= prediction_data['date_end']]
prediction_data['is_discount'] = ~prediction_data['promo_typeCode'].isna()
prediction_data['sale_price_before_promo'].fillna(prediction_data['sale_price_before_promo'].mean(), inplace=True)
prediction_data['sale_price_time_promo'].fillna(prediction_data['sale_price_time_promo'].mean(), inplace=True)
prediction_data['promo_typeCode'].fillna(0, inplace=True)

# Прогнозирование
X_pred = prediction_data[['item_id', 'day_of_week', 'is_discount', 'sale_price_before_promo', 'sale_price_time_promo']]
prediction_data['prediction'] = model.predict(X_pred)

# Подготовка данных для сохранения
predictions = prediction_data[['date', 'item_id', 'prediction']]

# Сохранение результатов в файл predictions.csv
predictions.to_csv('./predictions.csv', index=False, sep=';')
print("Файл predictions.csv успешно сохранен.")

RMSE на данных за декабрь 2023 года: 12.013152306227044
Файл predictions.csv успешно сохранен.
